# LoRa Mesh Radio Module Protocol Overview and Testbed
**Spencer Park, March 2019**

---

See https://www.dfrobot.com/product-1670.html which is also available from Digi key https://www.digikey.ca/product-detail/en/dfrobot/TEL0117/TEL0117-ND/8122310. 

The closest module is the `YL-800N` from `rf-modules.cn` which is the a lower frequency band but appears to support the same set of commands. The english version of the documentation is avaliable on github at https://github.com/Arduinolibrary/LoRa_Wireelss_Modules_YL but the `TEL0117` is not exactly the same. One major difference being that `AT` commands do not appear to be supported, only the hex format which is not documented. 

`YL-800T` is a similar transciever with the same identification number. http://www.rf-module.cn/updow/2015103016576135.pdf

There are many layers to the communication protocol but we can generalize to 2:

1.  The _controller_ layer. This level is for interactions with the module from some controller like this notebook or an arduino.
2.  The _application data_ layer. This layer is very similar to a tcp stack. In this level the nodes may be communicating with each other by passing these messages around. They are handled by the firmware.

This notebook will focus on the controller layer and more specifically the protocol for communicating with the module. 

The testbed is located towards the end of this document. It requires all proceeding protocol related definitions.

# Controller Protocol

### Example frame

#### Header

| **Size (bytes)** | 1 | 1 | 1 | 1 | N | 1 |
|------------------|---|---|---|---|---|---|
| **Description**  | Frame type | Frame number | Command type | Payload length | Payload | Validation |

#### Payload

| **Size (bytes)** | 2 | 1 | 1 | 1 | N | Variable |
|------------------|---|---|---|---|---|----------|
| **Description**  | Target Addr | ACK request | Send radius | Routing param | Data length | User data |

Data example: `05` `00` `01` `0A` `02 00` `00` `07` `01` `04` `12 34 56 78` `01`

* Frame type: 05 means to send user data
* Frame number: 00 (currently unused and never changes)
* Command type: 01 indicates application data sending request
* Load length: 0A refers to the number of bytes from the length of the load to the number of bytes before the test

* Target Addr: 02 00 indicates that the destination address sent is 00 02
* ACK request: 00 means no ACK response required
* Send radius: 07 means maximum 7 hops
* Discovery routing parameters: 01 indicates the way of automatic routing. The user does not need to intervene in the networking process.
* Data length: 04 means the user has 4 bytes of data to send.
* User data: 12 34 56 78 four bytes

* Validation: 01 XOR Validation Value

## Communication Primitives

All of the protocol classes communicate over an abstract `Socket` defined here for use in the rest of the notebook. 

They may use the write methods `put_*` to send data over the socket or read via the `get_*` methods to receive data. 

* **`get_byte() -> int` and `put_byte(int)`**: reading/writing a single byte.
* **`get_short() -> int` and `put_short(int)`**: reading/writing a short (2 bytes).
* **`get_array() -> bytes` and `put_array(bytes)`**: reading/writing a variable length sequence of bytes. This includes prefixing the array with a single byte describing the length of the array in bytes.
* **`get_bytes(int) -> bytes` and `put_bytes(bytes)`**: reading/writing a sequence of bytes. Since the length is static it must be specified when reading. When writing, the length of the `bytes` determines the number of bytes to write.

In [ ]:
from functools import reduce
from queue import Queue

import abc

class Socket(abc.ABC):
    def __init__(self):
        self._check = 0x00
        
    @abc.abstractmethod
    def _read(self, size=1):
        # (int) -> bytes
        pass
    
    @abc.abstractmethod
    def _write(self, data):
        # (bytes) -> Any
        pass
        
    def reset_checksum(self):
        self._check = 0x00
        
    def get_checksum(self):
        return self._check
    
    def check(self):
        return self._check == 0
    
    def _add_checksum_bytes(self, b):
        self._check = reduce(int.__xor__, b, self._check)
        return b
    
    def _write_checked(self, b):
        self._write(self._add_checksum_bytes(b))
        
    def _read_checked(self, size):
        return self._add_checksum_bytes(self._read(size=size))
        
    def put_byte(self, v):
        self._write_checked(v.to_bytes(1, byteorder='big'))
        
    def put_short(self, v):
        self._write_checked(v.to_bytes(2, byteorder='big'))
        
    def put_array(self, data):
        l = len(data)
        self.put_byte(l)
        self._write_checked(data)
        
    def put_bytes(self, data):
        self._write_checked(data)
        
    def get_byte(self):
        return int.from_bytes(self._read_checked(size=1), byteorder='big')
    
    def get_short(self):
        return int.from_bytes(self._read_checked(size=2), byteorder='big')
    
    def get_array(self):
        l = self.get_byte()
        data = self._read_checked(size=l)
        assert len(data) == l
        return data
    
    def get_bytes(self, size):
        return self._read_checked(size=size)

class BytesBackedSocket(Socket):
    def __init__(self, buf):
        super(BytesBackedSocket, self).__init__()
        self.buf = buf
        
    # @Override
    def _read(self, size=1):
        if len(self.buf) >= size:
            data = self.buf[0:size]
            self.buf = self.buf[size:]
            return data
        else:
            raise Exception('Not enough data available')
            
    # @Override
    def _write(self, d):
        self.buf = b''.join([self.buf, d])

# a queue of data fragments (bytes)
class BufferedBlockingQueueSocket(Socket):
    def __init__(self, queue, write, timeout=None):
        super(BufferedBlockingQueueSocket, self).__init__()
        self.queue = queue
        self.write = write
        self.timeout = timeout
        self._read_buffer = bytes()
    
    # @Override
    def _read(self, size=1):
        buffer = self._read_buffer
        while len(buffer) < size:
            segment = self.queue.get(block=True, timeout=self.timeout)
            buffer = buffer + segment
            
        data = buffer[0:size]
        self._read_buffer = buffer[size:]
        
        return data
    
    # @Override
    def _write(self, d):
        self.write(d)

In [ ]:
def test_queue():
    import queue

    q = queue.Queue()

    s = BufferedBlockingQueueSocket(q, lambda d: print(d.hex()))

    import threading
    import time

    def run():
        q.put(b'\x12')
        time.sleep(3)
        q.put(b'\x34')

    t = threading.Thread(target=run)
    t.start()

    s.put_short(0x5678)
    s.get_short().to_bytes(2, byteorder='big')
# test_queue()

## General frame format

| 1 | 1 | 1 | 1 | Var | 1 |
|:---|:---|:---|:---|:---|:---|
| frame type | frame number | command type | payload length | payload data | frame check |
| Frame header | | | | Frame payload  | Frame tail |

### Frame header

#### Frame type

The frame type is used to identify different application frame types. The standard frame types are defined as follows:

| Type | Name | Description|
|:---|:---|:---|
| `0x01` | Module configuration | Configuration parameters for reading and writing modules, etc. |
| `0x02` | MAC layer test | is used to test the networking protocol MAC layer function. |
| `0x03` | NET layer test | is used to test networking protocol network layer function |
| `0x04` | Debug information | Used to set or read some debugging test information, etc. |
| `0x05` | Application data | Used by the networking protocol application layer to use the interface. |

#### Frame number

The frame sequence number field is currently unused and the value is fixed at `0x00`.

#### Command type

The command type field has different definitions under various frame type identifiers. It can be thought of as a *sub frame type*.

####  Payload length

The load length field indicates the length of the frame payload portion of the upper frame format, that is, from the domain to the frame check

The number of bytes in the previous part. The maximum load length of this protocol is 128 bytes.

### Frame payload

The frame data specific to the header-specified frame type and command type. They are outlined later in the document.

### Frame tail

The frame tail field is a 1-byte XOR check. It is computed as the XOR of all bytes in the frame (excluding the tail). 

When verifying the frame, the XOR of all bytes in the frame _including the tail byte_ must be 0. Otherwise the verification fails.

## LoRaFrame and LoRaConnection

A `LoRaFrame` is a general frame object with an arbitrary `payload`. The static method `LoRaFrame.from_payload(payload)` is used to lift a payload object into a frame which can be sent over a `LoRaConnection`.

Payload classes should register themselves via the decorator `LoRaConnection.registered_payload_for(frame_type, cmd_type)` which will register the class for deserialization of incoming frames as well as set 2 static attributes `FRAME_TYPE` and `CMD_TYPE` on the class for use in `LoRaFrame.from_payload(payload)`.

A registered payload class **must** implement a static method `read_from_socket(Socket)` which reads the payload data from the given socket and returns an instance of that payload class. Likewise, payload classes must implement a method `write_to_socket(Socket)` which writes the data from the payload instance to the socket.

In [ ]:
from collections import namedtuple

LoRaFrame = namedtuple('LoRaFrame', 'frame_type frame_num cmd_type payload')
setattr(LoRaFrame, 'from_payload', 
        lambda payload: LoRaFrame(
            frame_type=payload.FRAME_TYPE, 
            frame_num=0x00,
            cmd_type=payload.CMD_TYPE,
            payload=payload
        ))

class LoRaConnection:
    PAYLOAD_REGISTRY = {}
    
    @staticmethod
    def registered_payload_for(frame_type, cmd_type):
        def decorator(deco_cls):
            setattr(deco_cls, 'FRAME_TYPE', frame_type)
            setattr(deco_cls, 'CMD_TYPE', cmd_type)
            LoRaConnection.PAYLOAD_REGISTRY.setdefault(frame_type, {})[cmd_type] = deco_cls
            
            return deco_cls
        
        return decorator
    
    @staticmethod
    def lookup_payload_type(frame_type, cmd_type):
        return LoRaConnection.PAYLOAD_REGISTRY.get(frame_type, {}).get(cmd_type, None)
    
    def __init__(self, socket):
        self.socket = socket
    
    def read_frame(self):
        self.socket.reset_checksum()
        
        frame_type = self.socket.get_byte()
        frame_num = self.socket.get_byte()
        cmd_type = self.socket.get_byte()
        
        payload = self.socket.get_array()
        
        PayloadCls = LoRaConnection.lookup_payload_type(frame_type, cmd_type)
        if PayloadCls is not None:
            payload_socket = BytesBackedSocket(payload)
            payload = PayloadCls.read_from_socket(payload_socket)
        
        check_byte = self.socket.get_byte()
        if not self.socket.check():
            raise Exception('Checksum failed')
            
        return LoRaFrame(frame_type, frame_num, cmd_type, payload)
    
    def write_frame(self, frame):
        self.socket.reset_checksum()
        self.socket.put_byte(frame.frame_type)
        self.socket.put_byte(frame.frame_num)
        self.socket.put_byte(frame.cmd_type)
        
        if hasattr(frame.payload, 'write_to_socket'):
            payload_socket = BytesBackedSocket(bytes())
            frame.payload.write_to_socket(payload_socket)
            self.socket.put_array(payload_socket.buf)
        else:
            self.socket.put_array(frame.payload)
        
        self.socket.put_byte(self.socket.get_checksum())


## Configuration

The configuration format is used for reading and writing. It is defined as follows and implemented by the `Configuration` class:

| 2 | 1 | 1 | 1 | 1 | 2 | 2 | 2 | 1 | 1 | 2 |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| Configuration flag | Channel number | RF transmit power | User interface mode | Equipment type | Network identification | Node identifier | Reserved (`0x0000`) | Reserved (`0x03`) | Serial port parameter | Air rate |

### Configuration flag (`flag`) - factory default: `0xA5A5`

A magic value that is fixed at `0xA5A5`.

### Channel number (`channel`) - factory default: `0x01`

The working channel number of the wireless module ranges from 0 to 7.

| **Channel number** | `0x00` | `0x01` | `0x02` | `0x03` | `0x04` | `0x05` | `0x06` | `0x07` |
|:---|:---|:---|:---|:---|:---|:---|:---|:---| 
| **Frequency (Hz)** | 431M | 432M | 429M | 433M | 436M | 434M | 437M | 435M |

### RF transmit power (`tx_power`) - factory default: `0x00`

The default is the maximum power. The convention with power seems to be lower $\rightarrow$ stronger.

### User interface mode (`ui_mode`) - factory default: `0x00`

The working mode of the module serial port interacting with the user. 

| `0x00` | `0x01` |
|:---|:---|
| hexadecimal command mode | transparent mode |

In **hexadecimal command mode**, all data received on the serial port is processed as a command. In **transparent mode**, all data received is treated as application data (data to be transmitted) and is dispatched according to the network configuration outlined below. 

Transparent mode can be used for a device that is mainly being used to send/receive data (after it is already configured). When in this mode, a command may be sent if prefixed with `+++`.

When the master module is in transparent mode, the data received by the serial port is transparently sent to all slave modules. When the slave module is in transparent mode, the data received by the serial port is transparently sent to a main module.

### Equipment type (`equip_type`) - factory default: `0x01`<sup>\*</sup> 
<sup>\* The document mentions a default value of `0x00` but in practice this has been `0x01`.</sup>

The role of the device within the network.

| `0x00` | `0x01` |
|:---|:---|
| slave device (node module) | master device (root module) |

### Network identification (`net_id`) - factory default: `0x0000`

The id of the network the device belongs to. Also refered to as a PAN (personal area network) id. Devices must belong to the same network to communicate. The value of the network identifier ranges from `0x0000` to `0xFFFE`.

### Node identifier (`node_id`) - factory default: `0x0600` <sup>\*</sup>
<sup>\* The document mentions that the lower 2 bytes of the serial number are the default node ID but in practice, all nodes had a default ID of `0x0600`</sup>

The address or id of the device within the network. Nodes should have unique identifiers within the network. Valid addresses are in the range `0x0000` to `0xFFFE` with `0xFFFF` reserved as the _broadcast_ address.

### Serial port parameter (`ser_flags`) - factory default: `0x40`<sup>\*</sup>
<sup>\* The document mentions defaults that in no way could correspond to a value of `0x40`. The defaults specified here are based on the `0x40` seen in practice.</sup>

The operating parameters of the module's serial port. The default `0x40` corresponds to the `9600/8N1` common serial port configuration that `SoftwareSerial` (and many others) are using. **Changing it is likely unnecessary**.

The bit order was not specified but based on what works, bit _1_ is on the right. The table was rearranged to support that. The 4 most significant bits describe the baudrate, the next 1 is reserved (and 0), the next 2 describe the parity, and the last bit describes the number of stop bits.

| **Bits** | 4 | 1 | 2 | 1 |
|:---|:---|:---|:---|:---|
| | Baudrate | Reserved (`0`) | Parity | Stop bits |  

**Baudrate** - factory default: `0x4`

| `0x1` | `0x2` | `0x3` | `0x4` | `0x5` | `0x6` | `0x7` | `0x8` | `0x9` | `0xA` | `0xB` | `0xC` |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| 1200 | 2400 | 4800 | 9600 | 14400 | 19200 | 28800 | 38400 | 57600 | 76800 | 115200 | 230400 |

**Parity** - factory default: `0`

| `0` | `1` | `2` |
|:---|:---|:---|
| none | odd | even |

**Stop bits** - factory default: `0` 

| `0` | `1` |
|:---|:---|
| 1 stop bit | 2 stop bits |

### Air rate (determined by signal bandwidth and spreading factor) (`air_rate`)- factory default: `0x0909`

The factory spread spectrum factor and signal bandwidth are 9, which is equivalent to the module's air modulation rate of 7K. **It is recommended that customers do not modify it**. Due to the large delay of the networking module, if the customer needs to calculate the air rate is greater than 7K, if the transmission is too low, the transmission and reception delay will be too large, which will affect the internal protocol of the network, resulting in network instability.

## Network configurations

### 1.  Fully peer-to-peer non-transparent networking mode

All nodes are in _hexadecimal command mode_ (`ui_mode` = `0x00`). Worked with both nodes configured as _master devices_ (`equip_type` = `0x01`) but unsure if this is a requirement.

There is no master-slave relationship between nodes, completely peer-to-peer, pure MESH network, all node equiptment types are the same. In this networking mode, all nodes can send data via hexadecimal commands. Broadcast via the `0xFFFF` target address. Unicast via the node identifier of the target. The receive indication payload also carries the source address of the data received. 
    
### 2.  Fully peer-to-peer transparent networking mode

All nodes are in _transparent mode_ (`ui_mode` = `0x01`). All nodes must be configured as _master devices_ (`equip_type` = `0x01`).

There is no master-slave relationship between all nodes, completely peer-to-peer, pure transparent transmission network, which can completely replace the common transparent transmission module on the market. In this networking mode, each node receives data from the serial port and transparently broadcasts it to all other modules. The module receives data sent by other modules as pure transparent application data. **Users can only broadcast data**.

### 3.  Master-slave non-transparent networking mode

All nodes are in _hexadecimal command mode_ (`ui_mode` = `0x00`). Single _master device_ (`equip_type` = `0x01`) and rest _slave devices_ (`equip_type` = `0x00`).

Identical to the first mode, fully peer-to-peer non-transparent networking mode.
    
### 4.  Master-slave transparent networking mode

All nodes are in _transparent mode_ (`ui_mode` = `0x01`). Single _master device_ (`equip_type` = `0x01`) and rest _slave devices_ (`equip_type` = `0x00`).

In this networking mode, the data received by the serial port of the master node is transparently broadcasted to the slave nodes, and the data received from the serial port of the slave nodes is transparently unicast to the master node. 
    
### 5.  Master-slave translucent networking mode

The master node is in _hexadecimal command mode_ (`ui_mode` = `0x00`) and all slave nodes are in _transparent mode_ (`ui_mode` = `0x01`). Single _master device_ (`equip_type` = `0x01`) and rest _slave devices_ (`equip_type` = `0x00`).

In this networking mode, the master node can send broadcasts to all slave nodes as well as unicast data to a specific slave node. Data received on the serial port of a slave node will be unicast to the master node. 

In [ ]:
class Configuration(namedtuple('Configuration', 'flag channel tx_power ui_mode equip_type net_id node_id resv ser_flags air_rate')):
    @staticmethod
    def factory_default(**kw):
        defaults = {'flag': 0xA5A5, 'channel': 0x01, 'tx_power': 0x00, 'ui_mode': 0x00, 'equip_type': 0x01,
                   'net_id': 0x0000, 'node_id': 0x0600, 'resv': b'\x00\x00\x03', 'ser_flags': 0x40, 'air_rate': 0x0909}
        defaults.update(kw)
        return Configuration(**defaults)
    
    def extend(self, **kw):
        values = self._asdict()
        values.update(kw)
        return Configuration(**values)
    
    @staticmethod
    def read_from_socket(socket):
        flag = socket.get_short()
        assert flag == 0xA5A5
        
        channel = socket.get_byte()
        tx_power = socket.get_byte()
        ui_mode = socket.get_byte()
        equip_type = socket.get_byte()
        
        net_id = socket.get_short()
        node_id = socket.get_short()
        
        resv = socket.get_bytes(3)
        
        ser_flags = socket.get_byte()
        air_rate= socket.get_short()
        
        return Configuration(flag, channel, tx_power, ui_mode, equip_type, net_id, node_id, resv, ser_flags, air_rate)
    
    def write_to_socket(self, socket):
        socket.put_short(self.flag)
        
        socket.put_byte(self.channel)
        socket.put_byte(self.tx_power)
        socket.put_byte(self.ui_mode)
        socket.put_byte(self.equip_type)
        
        socket.put_short(self.net_id)
        socket.put_short(self.node_id)
        
        socket.put_bytes(self.resv)
        
        socket.put_byte(self.ser_flags)
        socket.put_short(self.air_rate)

## Frame payloads

### Module configuration frame type (`0x01`)

The module configuration frame type is identified as `0x01`. The command types under the module configuration type are defined as follows:

| Command Type | Description |
|---:|:---|
| `0x01` | Write configuration information request |
| `0x81` | Write configuration information response |
| `0x02` | Read configuration information request |
| `0x82` | Read configuration information response |
| `0x06` | Read version information request |
| `0x86` | Read version information response |
| `0x07` | Module reset request |
| `0x87` | Module reset response |

#### Write configuration information request (`0x01`)

The write config request is used to change the configuration of the module. The payload is a complete configuration (see the above [configuration section](#Configuration). Copied here for convience but the testbed implementation uses an instance of `Configuration`:

| 2 | 1 | 1 | 1 | 1 | 2 | 2 | 2 | 1 | 1 | 2 |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| Configuration flag | Channel number | RF transmit power | User interface mode | Equipment type | Network identification | Node identifier | Reserved (`0x0000`) | Reserved (`0x03`) | Serial port parameter | Air rate |

#### Write configuration information response (`0x81`)

The write config response is returned after a write config request is issued.

The payload is defined as follows:

| 1 |
|:---|
| Return status |

##### Return status

The status code meanings are as follows with anything but `0x00` indicating an error:

| Code | Description |
|---:|:---|
| `0x00` | Success |
| `0x01` | XOR check error |
| `0x02` | Test frame transmission error |
| `0x03` | Command error |
| `0x04` | Information setting error |
| `0x05` | Length error |
| `0x06` | Write flash failed error |

In [ ]:
@LoRaConnection.registered_payload_for(0x01, 0x01)
class WriteConfigReq(namedtuple('ReadConfigReq', 'config')):
    @staticmethod
    def read_from_socket(socket):
        config = Configuration.read_from_socket(socket)
        return WriteConfigReq(config)
    
    def write_to_socket(self, socket):
        self.config.write_to_socket(socket)
    
@LoRaConnection.registered_payload_for(0x01, 0x81)
class WriteConfigResp(namedtuple('WriteConfigResp', 'status')):
    STATUS_CODES = dict(enumerate([
        'SUCCESS', 'XOR_CHECK_ERROR', 'TEST_FRAME_TX_ERROR', 'COMMAND_ERROR', 
        'INFO_SETTING_ERROR', 'LENGTH_ERROR', 'WRITE_FLASH_ERROR'
    ]))
    
    @staticmethod
    def read_from_socket(socket):
        status = socket.get_byte()
        return WriteConfigResp(status)
    
    def write_to_socket(self, socket):
        socket.put_byte(self.status)
        
    def __str__(self):
        return 'WriteConfigResp(%s(0x%02x))' % (WriteConfigResp.STATUS_CODES.get(self.status, 'UNKNOWN_CODE'), self.status)

In [ ]:
str(WriteConfigResp(0x01))

#### Read configuration information request (`0x02`)

The read config request is used to read the configuration parameters stored in the module. The payload is empty.

#### Read configuration information response (`0x82`)

The read configuration information response payload is a complete configuration (see the above [configuration section](#Configuration). Copied here for convience but the testbed implementation uses an instance of `Configuration`:

| 2 | 1 | 1 | 1 | 1 | 2 | 2 | 2 | 1 | 1 | 2 |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| Configuration flag | Channel number | RF transmit power | User interface mode | Equipment type | Network identification | Node identifier | Reserved (`0x0000`) | Reserved (`0x03`) | Serial port parameter | Air rate |

In [ ]:
@LoRaConnection.registered_payload_for(0x01, 0x02)
class ReadConfigReq(namedtuple('ReadConfigReq', '')):
    @staticmethod
    def read_from_socket(socket):
        return ReadConfigReq()
    
    def write_to_socket(self, socket):
        pass
    
@LoRaConnection.registered_payload_for(0x01, 0x82)
class ReadConfigResp(namedtuple('ReadConfigReq', 'config')):
    @staticmethod
    def read_from_socket(socket):
        config = Configuration.read_from_socket(socket)
        return ReadConfigResp(config)
    
    def write_to_socket(self, socket):
        self.config.write_to_socket(socket)

#### Read version information request (`0x06`)

The read version request is used to read the version information of the firmware in the module. The payload is empty.

#### Read version information response (`0x86`)

The read version information response is the firmware version information returned by the module after receiving the read version information request command. The payload is as follows:

| 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 
|:---|:---|:---|:---|:---|:---|:---|:---|
| Major version number | Minor version number | Revision number | Hardware type code | Compilation date day | Compilation date month | Compilation date year | Equipment type |

**Note**: The current hardware type code field and device type field are left unused.

In [ ]:
import datetime

@LoRaConnection.registered_payload_for(0x01, 0x06)
class ReadVersionInfoReq(namedtuple('ReadVersionInfoReq', '')):
    @staticmethod
    def read_from_socket(socket):
        return ReadVersionInfoReq()
    
    def write_to_socket(self, socket):
        pass
    
@LoRaConnection.registered_payload_for(0x01, 0x86)
class ReadVersionInfoResp(namedtuple('ReadVersionInfoResp', 'major minor rev compilation_date')):
    @staticmethod
    def read_from_socket(socket):
        major, minor, rev = (socket.get_byte() for _ in range(3))
        hardware_type_code = socket.get_byte()
        day, month, year = (socket.get_byte() for _ in range(3))
        equiptment_type = socket.get_byte()
        return ReadVersionInfoResp(major, minor, rev, datetime.date(year, month + 1, day + 1))
    
    def write_to_socket(self, socket):
        socket.write_byte(self.major)
        socket.write_byte(self.minor)
        socket.write_byte(self.rev)
        
        socket.write_byte(0x00) # hardware_type_code
        
        socket.write_byte(self.compilation_date.day)
        socket.write_byte(self.compilation_date.month - 1)
        socket.write_byte(self.compilation_date.year - 1)
        
        socket.write_byte(0x00) # equiptment_type
        
    def __str__(self):
        return 'ReadVersionInfoResp(v%d.%d-rev %d compiled %s)' % (self.major, self.minor, self.rev, str(self.compilation_date))

####  Module reset request (`0x07`)

The module reset request is used to soft reset the wireless module. The payload is empty.

#### Module reset response (`0x87`)

The module reset response is the execution status information returned by the module after receiving the module reset request command.

The payload is defined as follows:

| 1 |
|:---|
| Return status |

##### Return status

The status code meanings are as follows with anything but `0x00` indicating an error:

| Code | Description |
|---:|:---|
| `0x00` | Success |
| `0x01` | XOR check error |
| `0x02` | Test frame transmission error |
| `0x03` | Command error |
| `0x04` | Information setting error |
| `0x05` | Length error |
| `0x06` | Write flash failed error |

**Note**: When the command is executed, the module is reset and the external module does not receive the module reset response so don't wait for it.

In [ ]:
@LoRaConnection.registered_payload_for(0x01, 0x07)
class ModuleResetReq(namedtuple('ModuleResetReq', '')):
    @staticmethod
    def read_from_socket(socket):
        return ModuleResetReq()
    
    def write_to_socket(self, socket):
        pass
    
@LoRaConnection.registered_payload_for(0x01, 0x87)
class ModuleResetResp(namedtuple('ModuleResetResp', 'status')):
    STATUS_CODES = dict(enumerate([
        'SUCCESS', 'XOR_CHECK_ERROR', 'TEST_FRAME_TX_ERROR', 'COMMAND_ERROR', 
        'INFO_SETTING_ERROR', 'LENGTH_ERROR', 'WRITE_FLASH_ERROR'
    ]))
    
    @staticmethod
    def read_from_socket(socket):
        status = socket.get_byte()
        return ModuleResetResp(status)
    
    def write_to_socket(self, socket):
        socket.put_byte(self.status)
        
    def __str__(self):
        return 'ModuleResetResp(%s(0x%02x))' % (ModuleResetResp.STATUS_CODES.get(self.status, 'UNKNOWN_CODE'), self.status)

### Application data frame type (`0x05`)

The identifier of the application data type is `0x05`. The command types under the application data type are defined as follows:

| Command type identifier | Command function description |
|---:|:---|
| `0x01` | Application data send request |
| `0x81` | Application data send response |
| `0x82` | Application data reception indication |
| `0x08` | Source route discovery request |
| `0x88` | Source route discovery response |

#### Application data send request (`0x01`)

The application data sending request command is used by an external device to send data through the wireless networking module. It is a request for the lora module to send data.

The payload is as follows:

| 2 | 1 | 1 | 1 | 1 | N\*2 | 1 | Var |
|:---|:---|:---|:---|:---|:---|:---|:---|
| Target address | Application layer ACK request | Send radius | Discover routing parameters | Number of relays N | Relay list | Application service data unit length | Application business data unit |

##### Target address (`target_addr`)

The 2-byte short address (lower byte first) of the data transmission destination node, or `0xFFFF` to broadcast to all reachable nodes.
    
##### Application layer ACK request (`ack`)

When `0x01`, the end-to-end acknowledgment retransmission mechanism of the protocol APS layer is used. If it is `0x00`, it is not used. It is recommended not to use this feature when responding to the application layer peer application device.
    
##### Send radius (`send_radius`)

Maximum number of hops for data forwarding. The maximum hop value of the current networking protocol is reported to be `0x07` (and somehow customizable?). Recommended to just used `0x07` unless a more specific restriction is necessary.
    
##### Route discovery parameters (`route_discovery`)

For general purpose use in a mesh network, `0x02` (automatic discorvery), is recommended.

| **Value** | **Description** |
|:----------|:----------------|
| `0x00` | Route discovery is disabled. If there is no route to the destination node in the routing table, the transmission fails. |
| `0x01` | Automatic route discovery. if there is a route to the destination node in the routing table use it, otherwise automatically find a route. |
| `0x02` | Forced route discovery. Look for new routes regardless of whether there is a route to the destination node in the routing table. |
| `0x03` | Use source routing. |
    
##### Source routing domain (`relays`) <sup>*</sup>
<sup>*This section is only present if `route_discovery` is `0x03`</sup>

If the `route_discovery` mode is `0x03` (source routing mode), then the route is specified. The route is an array of 2-byte short addresses describing all intermediate (excluding source and destination) hops.

The first part is the number of relays `N`. It is the number of intermediate nodes in the relay list and must be in the range 0-6.

The second part is the variable length relay list. It contains the 2-byte short addresses (lower byte first) of the relay nodes in the path from the destination node to the source node. The node address nearest the destination is comes first and subsequent address get "closer" to the source address.

##### Application business data (`payload`)

This is the application (or user) data to send. The data is a byte array and as such starts with a single byte describing the length (in bytes) followed by the contents.

Since the current physical layer has a maximum load length of 127, the maximum payload length is 111 as a result of all of the headers. When using source routing, the maximum length is 109-N\*2 (N is the number of relays).

In [ ]:
@LoRaConnection.registered_payload_for(0x05, 0x01)
class AppDataSendReq(namedtuple('AppDataSendReq', 'target_addr ack send_radius route_discovery relays payload')):
    BROADCAST_ADDR = 0xFFFF
    
    class RouteDiscovery:
        DISABLED = 0x00
        AUTOMATIC = 0x01
        FORCED = 0x02
        RELAYS_PROVIDED = 0x03
    
    @staticmethod
    def read_from_socket(socket):
        target_addr = socket.get_short()
        ack = socket.get_byte()
        send_radius = socket.get_byte()
        route_discovery = socket.get_byte()
        if route_discovery == AppDataSendReq.RouteDiscovery.RELAYS_PROVIDED:
            relays = list(map(lambda x: socket.get_short(), range(socket.get_byte())))
        else:
            relays = []
        payload = socket.get_array()
        
        return AppDataSendReq(target_addr, ack, send_radius, route_discovery, relays, payload)
    
    def write_to_socket(self, socket):
        socket.put_short(self.target_addr)
        socket.put_byte(self.ack)
        socket.put_byte(self.send_radius)
        socket.put_byte(self.route_discovery)
        if self.route_discovery == AppDataSendReq.RouteDiscovery.RELAYS_PROVIDED:
            socket.put_byte(len(self.relays))
            for r in self.relays:
                socket.put_short(r)
        socket.put_array(self.payload)

#### Application data send response (`0x81`)

After the module receives the application data transmission request, it will reply to the external reply after executing the data transmission. Its payload is defined as follows:

| 2 | 1 |
|:---|:---|
| target address | return status |

##### Target address (`target_addr`)

The target node address in the corresponding application data transmission request.

##### Return status (`status`)

The return status indicates the execution result of the transmission. The values of the return status are defined as follows:

| Return status value | Definition |
|---:|:---|
| `0x00` | success |
| `0xE1` | XOR check error |
| `0xE4` | security check failed |
| `0xE5` | MAC frame long error |
| `0xE6` | invalid parameter |
| `0xE7` | did not receive ACK |
| `0xEA` | transmitter is busy |
| `0xC1` | network layer invalid parameter |
| `0xC2` | invalid request |
| `0xC7` | no route found |
| `0xD1` | buffer busy |
| `0xD2` | APS layer did not receive ACK |
| `0xD3` | APS frame is too long |

`0xE*` are error codes for the MAC layer, `0xC*` are error codes for the network layer, and `0xD*` are error codes for the APS layer.

**Note**: In the data transmission without the APS layer ACK request mechanism, even if the return code is successful, it does not mean that the data is successfully transmitted to the target address, but only means that the data is normally sent to the next hop.

In [ ]:
from functools import reduce

@LoRaConnection.registered_payload_for(0x05, 0x81)
class AppDataSendResp(namedtuple('AppDataSendResp', 'target_addr status')):
    MAC_LAYER_ERROR_CODES = {
        0xE1: 'XOR checksum error',
        0xE4: 'Security check failed',
        0xE5: 'MAC frame too long',
        0xE6: 'Invalid parameter',
        0xE7: 'Did not receive ACK',
        0xEA: 'Transmitter busy',
    }
    
    NETWORK_LAYER_ERROR_CODES = {
        0xC1: 'Invalid parameter',
        0xC2: 'Invalid request',
        0xC7: 'No route found',
    }
    
    APS_LAYER_CODES = {
        0xD1: 'Buffer busy',
        0xD2: 'Did not receive ACK',
        0xD3: 'APS frame too long'
    }
    
    STATUS_CODES = reduce(lambda all_codes, codes: all_codes.update(codes) or all_codes, [
        {c: ('MACLayer::%s' % desc) for c, desc in MAC_LAYER_ERROR_CODES.items()},
        {c: ('NetworkLayer::%s' % desc) for c, desc in NETWORK_LAYER_ERROR_CODES.items()},
        {c: ('APSLayer::%s' % desc) for c, desc in APS_LAYER_CODES.items()},
    ], {0x00: 'Success'})
    
    @staticmethod
    def read_from_socket(socket):
        target_addr = socket.get_short()
        status = socket.get_byte()
        
        return AppDataSendResp(target_addr, status)
    
    def write_to_socket(self, socket):
        socket.put_short(self.target_addr)
        socket.put_short(self.status)
        
    def is_success(self):
        return self.status == 0x00
        
    def __str__(self):
        return 'AppDataSendResp(target_addr=0x%04x, status=0x%02x(%s))' \
            % (self.target_addr, self.status, AppDataSendResp.STATUS_CODES.get(self.status, 'UNKNOWN_CODE'))

In [ ]:
str(AppDataSendResp(0x1234, 0xE4))

#### Application data reception indication (`0x02`)

This command is an unsolicited command sent from the LoRa module to the external device when the module receives data from the network.

**Note:** The document omits the _"Field strength"_ field from the hex command protocol description but includes it in an earlier example. The field was found to be present in practice while using the device mentioned at the top of this document.

The payload is as follows:

| 2 | 1 | 1 | Var |
|:---|:---|:---|:---|
| Source address | Field strength | Payload length | Payload |

##### Source address (`source_addr`)
    
The address of the sender of the data received.

##### Field strength (`strength`)

A number representing the strength of the receiving signal. The **smaller** the value, the **stronger** the signal.

##### Payload (`payload`)

The data that was received. It is a byte array and is therefore prefixed with the length of the following segment in bytes.

In [ ]:
@LoRaConnection.registered_payload_for(0x05, 0x82)
class AppDataRecv(namedtuple('AppDataRecv', 'source_addr strength payload')):
    BROADCAST_ADDR = 0xFFFF
    
    @staticmethod
    def read_from_socket(socket):
        source_addr = socket.get_short()
        strength = socket.get_byte()
        payload = socket.get_array()
        
        return AppDataRecv(source_addr, strength, payload)
    
    def write_to_socket(self, socket):
        socket.put_short(self.source_addr)
        socket.put_byte(self.strength)
        socket.put_array(self.payload)

In [ ]:
str(AppDataRecv(0x1234, 0x00, b'\x56\x78'))

# LoRa Management Tools

The remainder of the document is for testing the modules via [pySerial](https://github.com/pyserial/pyserial) and requires a USB to TTL converter with atleast a VCC, GNC, TXD, and RXD pin. The module should be connected to the laptop running the notebook server.

In [ ]:
from typing import Callable, Optional

import queue
import threading
import serial.threaded
    
class Connection:
    def __init__(self, port):
        self._callbacks_lock = threading.RLock()
        
        self._frame_callbacks = []
        self._open_callbacks  = []
        self._close_callbacks = []
        
        _data_queue = queue.Queue()
        _socket = BufferedBlockingQueueSocket(_data_queue, port.write)
        self._lora_conn = LoRaConnection(_socket)
        self._frame_queue = queue.Queue()
        
        self._frame_parser_alive = True
        closure = self
        class Protocol(serial.threaded.Protocol):
            def connection_made(self, transport):
                super(Protocol, self).connection_made(transport)
                with closure._callbacks_lock:
                    for cb in closure._open_callbacks:
                        cb()
            
            def connection_lost(self, exc):
                super(Protocol, self).connection_lost(exc)
                with closure._callbacks_lock:
                    for cb in closure._close_callbacks:
                        cb(exc)

            def data_received(self, data):
                _data_queue.put(data)
        
        def _parse_frames():
            while port.is_open:
                try:
                    frame = closure._lora_conn.read_frame()

                    consumed = False
                    with closure._callbacks_lock:
                        for cb in closure._frame_callbacks:
                            if cb(frame):
                                consumed = True
                                break
                    if not consumed:
                        closure._frame_queue.put(frame)
                except queue.Empty:
                    continue
            
        self._reader_thread = serial.threaded.ReaderThread(port, Protocol)
        self._frame_thread = threading.Thread(target=_parse_frames)
            
    def on_frame(self, cb: Callable[[str], None]):
        with self._callbacks_lock:
            self._frame_callbacks.append(cb)
        return cb  # for use as a decorator

    def on_open(self, cb: Callable[[], None]):
        with self._callbacks_lock:
            self._open_callbacks.append(cb)
        return cb

    def on_close(self, cb: Callable[[Optional[BaseException]], None]):
        with self._callbacks_lock:
            self._close_callbacks.append(cb)
        return cb
        
    def start(self):
        self._reader_thread.start()
        self._frame_thread.start()

    def stop(self):
        self._reader_thread.close()
        
    def send_frame(self, frame):
        self._lora_conn.write_frame(frame)
    
    def send_payload(self, payload):
        self.send_frame(LoRaFrame.from_payload(payload))
    
    def send_request(self, payload, reply_cls, timeout=None):
        self.send_payload(payload)
        
        try:
            frame = self._frame_queue.get(block=True, timeout=timeout)
            if isinstance(frame.payload, reply_cls):
                return frame
        except queue.Empty:
            return None
    
    def take_available_frames(self):
        frames = []
        
        while True:
            try:
                frame = self._frame_queue.get(block=False)
                frames.append(frame)
            except queue.Empty:
                break
                
        return frames

The LoRa mesh node should be connected to a usb port. The following cell will list the open serial ports. A port should be chosen from this list.

In [ ]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

LORA_PORT = None

if len(ports) == 0:
    print('No ports connected.')
else:
    print(*map(lambda p: '  * ' + str(p), ports), sep='\n')
    LORA_PORT = input('Type the name of the port connected to the LoRa module: ')
    for p in ports:
        if p.device == LORA_PORT: break
    else:
        print('Invalid port name.')
        LORA_PORT = None
    print('Using', LORA_PORT)

In [ ]:
import serial
import time

LORA_READ_TIMEOUT = 3.0                 # in seconds

LORA_BAUDRATE     = 9600                # default for the YL-800N
LORA_STOP_BITS    = serial.STOPBITS_ONE # default for the YL-800N
LORA_PARITY       = serial.PARITY_NONE  # default for the YL-800N
LORA_BYTESIZE     = serial.EIGHTBITS    # default for the YL-800N

lora_serial_port = serial.Serial(port=LORA_PORT, timeout=LORA_READ_TIMEOUT, 
                           baudrate=LORA_BAUDRATE, stopbits=LORA_STOP_BITS, 
                           parity=LORA_PARITY, bytesize=LORA_BYTESIZE)
try:
    if lora_serial_port.inWaiting() > 0:
        lora_serial_port.flushInput()
    
    conn = Connection(lora_serial_port)
    
    @conn.on_frame
    def on_recv(frame):
        print('Read frame:', frame)
        if isinstance(frame.payload, AppDataRecv):
            print('Recv application data:', str(frame.payload))
            return True # notify that this handler consumed the frame
        return False
    
    conn.start()
    
    try:
        # Edit code in this try block
        resp = conn.send_request(ReadConfigReq(), ReadConfigResp, timeout=3)
        if resp is None:
            print('Could not read config')
        else:
            config = resp.payload.config
            print('Config:', str(config))
            
        # keep the connection open for 12 sec to listen for data recv indications handled by the @conn.on_frame callback
        time.sleep(12)
    finally:
        conn.stop()

finally:
    lora_serial_port.close()

In [ ]:
import serial

LORA_READ_TIMEOUT = 3.0                 # in seconds

LORA_BAUDRATE     = 9600                # default for the YL-800N
LORA_STOP_BITS    = serial.STOPBITS_ONE # default for the YL-800N
LORA_PARITY       = serial.PARITY_NONE # was EVEN  # default for the YL-800N
LORA_BYTESIZE     = serial.EIGHTBITS    # default for the YL-800N

lora_serial_port = serial.Serial(port=LORA_PORT, timeout=LORA_READ_TIMEOUT, 
                           baudrate=LORA_BAUDRATE, stopbits=LORA_STOP_BITS, 
                           parity=LORA_PARITY, bytesize=LORA_BYTESIZE)


try:
    for i in range(10):
        print(lora_serial_port.read(100))
finally:
    lora_serial_port.close()

The `print_frame` function is for debugging payloads. It dumps the frame containing the payload as a sequence of bytes written in hex. This is useful for checking what to send on the arduino side which is missing the payload library.

In [ ]:
def print_frame(payload):
    socket = BytesBackedSocket(bytes())
    lora = LoRaConnection(socket)
    lora.write_frame(LoRaFrame.from_payload(payload))
    print(' '.join(list(map(hex, socket.buf))))
print_frame(AppDataSendReq(0x1024, 0x0, 0x7, 0x01, [], b'\x01\x02\x03\x04'))